In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2024-07-22


Found rundate 2024-07-21 10:00:02 < 2024-07-22 at commit d844bcd9; returning commit e87ec5d


'2024-07-22 10:00:04'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2024-07-22 10:00:04, NJSP was reporting 344 YTD deaths
Current YTD Deaths (2025-07-22 10:00:03-04:00): 296
Previous year YTD Deaths (adjusted; 2024-07-22 10:00:04): 343.24635571227174
Projected 2025 total: 612.5954034917343
-13.8% change


3d193f73b23ed36caf66e95d6363ea55c9132bd1: FAUQStats cache miss: 2024, Mon Jul 22 10:00:04 EDT 2024


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,HIGHWAY,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,STREET,INJURIES,dt
ACCID,,,,,,,,,,,,,,
12918,12,Middlesex,1218,Plainsboro Twsp,1,State Highway 1,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-01-01 01:56:00-05:00
12923,11,Mercer,1111,Trenton,129,State Highway 129,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-01-03 12:14:00-05:00
12932,01,Atlantic,0112,Hamilton Twsp,446,Atlantic City Expressway MP 20.5,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-01-05 09:14:00-05:00
12933,02,Bergen,0248,Ramsey Boro,NaN,East Oak St,1.0,1.0,0.0,0.0,0.0,East Oak St,NaN,2024-01-05 14:12:00-05:00
12927,20,Union,2009,Linden City,27,State Highway 27,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-01-06 08:37:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13308,20,Union,2002,Clark Twsp,NaN,Madison Hill Rd,1.0,1.0,0.0,0.0,0.0,Madison Hill Rd,NaN,2024-07-16 19:52:00-04:00
13310,18,Somerset,1802,Bernards Twsp,287,Interstate 287 MP 29.6,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-07-17 13:46:00-04:00
13309,09,Hudson,0906,Jersey City,NaN,Communipaw Ave,1.0,1.0,0.0,0.0,0.0,Communipaw Ave,NaN,2024-07-17 18:30:00-04:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,11,2,6,0,18
Bergen,8,1,4,0,13
Burlington,14,4,3,0,20
Camden,12,2,8,0,21
Cape May,5,1,0,0,6
Cumberland,6,2,3,0,10
Essex,4,3,13,1,21
Gloucester,10,4,2,0,13
Hudson,2,1,3,1,6


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

55.4% through the year, 44.6% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,15,31,11
Bergen,driver,10,14,8
Burlington,driver,19,31,14
Camden,driver,12,19,12
Cape May,driver,2,8,5
...,...,...,...,...
Salem,crashes,8,14,10
Somerset,crashes,9,18,7
Sussex,crashes,7,12,2


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,39,0,25,4,13
Bergen,27,0,12,2,12
Burlington,37,0,24,4,12
Camden,32,0,19,3,12
Cape May,21,0,16,1,0
Cumberland,23,0,15,6,8
Essex,42,1,11,10,25
Gloucester,38,0,24,13,6
Hudson,22,1,8,1,18


In [9]:
projected.to_csv(PROJECTED_CSV)